In [1]:
import pandas as pd
import numpy as np
import warnings

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',10)
warnings.simplefilter('ignore')

file_path = 'D:\\mydata\\데이터분석\\FastCampus_전처리\\part-5.-머신러닝-모델의-성능-향상을-위한-전처리\\5. 머신러닝 모델의 성능 향상을 위한 전처리\\데이터\\'

In [2]:
df = pd.read_csv(file_path+'Telco_churn_prediction.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [3]:
X = df.drop(['customerID','Churn'], axis=1)
Y = df['Churn']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y)

In [4]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5282 entries, 1787 to 2384
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5282 non-null   object 
 1   SeniorCitizen     5282 non-null   int64  
 2   Partner           5282 non-null   object 
 3   Dependents        5282 non-null   object 
 4   tenure            5282 non-null   int64  
 5   PhoneService      5282 non-null   object 
 6   MultipleLines     5282 non-null   object 
 7   InternetService   5282 non-null   object 
 8   OnlineSecurity    5282 non-null   object 
 9   OnlineBackup      5282 non-null   object 
 10  DeviceProtection  5282 non-null   object 
 11  TechSupport       5282 non-null   object 
 12  StreamingTV       5282 non-null   object 
 13  StreamingMovies   5282 non-null   object 
 14  Contract          5282 non-null   object 
 15  PaperlessBilling  5282 non-null   object 
 16  PaymentMethod     5282 non-null   objec

In [5]:
for col in x_train.columns:
    print(f'{col} : {sorted(x_train[col].unique())[:10]} ::: {len(x_train[col].unique())}')

gender : ['Female', 'Male'] ::: 2
SeniorCitizen : [0, 1] ::: 2
Partner : ['No', 'Yes'] ::: 2
Dependents : ['No', 'Yes'] ::: 2
tenure : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] ::: 73
PhoneService : ['No', 'Yes'] ::: 2
MultipleLines : ['No', 'No phone service', 'Yes'] ::: 3
InternetService : ['DSL', 'Fiber optic', 'No'] ::: 3
OnlineSecurity : ['No', 'No internet service', 'Yes'] ::: 3
OnlineBackup : ['No', 'No internet service', 'Yes'] ::: 3
DeviceProtection : ['No', 'No internet service', 'Yes'] ::: 3
TechSupport : ['No', 'No internet service', 'Yes'] ::: 3
StreamingTV : ['No', 'No internet service', 'Yes'] ::: 3
StreamingMovies : ['No', 'No internet service', 'Yes'] ::: 3
Contract : ['Month-to-month', 'One year', 'Two year'] ::: 3
PaperlessBilling : ['No', 'Yes'] ::: 2
PaymentMethod : ['Bank transfer (automatic)', 'Credit card (automatic)', 'Electronic check', 'Mailed check'] ::: 4
MonthlyCharges : [18.25, 18.4, 18.55, 18.7, 18.75, 18.8, 18.85, 18.9, 18.95, 19.0] ::: 1453
TotalCharges : [' ', '

In [6]:
x_train['TotalCharges'].replace(' ', np.nan, inplace=True)
x_test['TotalCharges'].replace(' ', np.nan, inplace=True)

x_train['TotalCharges'] = x_train['TotalCharges'].astype(float)
x_test['TotalCharges'] = x_test['TotalCharges'].astype(float)

In [7]:
category_cols = [col for col in x_train.columns if x_train[col].dtypes==object]
category_cols

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [12]:
def make_suffix(df):
    suffix_cols = []
    col_names = []
    for col in df.columns:
        values = sorted(df[col].unique())[1:]
        suffix_cols += values
        col_names += [col] * len(values)
        
    return list(pd.Series(col_names)+'_'+pd.Series(suffix_cols))

In [15]:
from sklearn.preprocessing import OneHotEncoder as OHE

dummy_model = OHE(drop='first', sparse=False).fit(x_train[category_cols])

x_train_ohe = pd.DataFrame(dummy_model.transform(x_train[category_cols]), columns=make_suffix(x_train[category_cols]))
x_test_ohe = pd.DataFrame(dummy_model.transform(x_test[category_cols]), columns=make_suffix(x_test[category_cols]))

x_train_ohe.index = x_train.index
x_test_ohe.index = x_test.index

x_train = pd.concat([x_train.drop(category_cols, axis=1), x_train_ohe], axis=1)
x_test = pd.concat([x_test.drop(category_cols, axis=1), x_test_ohe], axis=1)

In [16]:
x_train.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,...,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
1787,1,42,105.20,4599.15,0.0,...,0.0,1.0,0.0,0.0,0.0
4844,0,37,20.30,755.40,0.0,...,0.0,0.0,0.0,0.0,0.0
6338,0,1,20.25,20.25,1.0,...,0.0,1.0,0.0,0.0,1.0
626,0,23,84.25,1968.10,1.0,...,0.0,0.0,0.0,1.0,0.0
71,0,52,20.40,1090.65,0.0,...,0.0,1.0,0.0,1.0,0.0


In [20]:
from sklearn.impute import KNNImputer

KI = KNNImputer(n_neighbors=5)
KI.fit(x_train)

x_train = pd.DataFrame(KI.transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(KI.transform(x_test), columns=x_test.columns)
x_train.isna().sum()

SeniorCitizen                            0
tenure                                   0
MonthlyCharges                           0
TotalCharges                             0
gender_Male                              0
Partner_Yes                              0
Dependents_Yes                           0
PhoneService_Yes                         0
MultipleLines_No phone service           0
MultipleLines_Yes                        0
InternetService_Fiber optic              0
InternetService_No                       0
OnlineSecurity_No internet service       0
OnlineSecurity_Yes                       0
OnlineBackup_No internet service         0
OnlineBackup_Yes                         0
DeviceProtection_No internet service     0
DeviceProtection_Yes                     0
TechSupport_No internet service          0
TechSupport_Yes                          0
StreamingTV_No internet service          0
StreamingTV_Yes                          0
StreamingMovies_No internet service      0
StreamingMo

In [21]:
cont_cols = [col for col in x_train.columns if len(x_train[col].unique()) > 3]
cate_cols = [col for col in x_train.columns if len(x_train[col].unique()) <= 3]

cont_cols

['tenure', 'MonthlyCharges', 'TotalCharges']

In [23]:
def IQR_Rule(arr):
    q1 = np.quantile(arr, 0.25)
    q3 = np.quantile(arr, 0.75)
    IQR = q3-q1
    
    not_outliers_cond = (q1-1.5*IQR < arr)&(q3+1.5*IQR > arr)
    return not_outliers_cond

num_outliers = len(x_train) - x_train[cont_cols].apply(IQR_Rule).sum(axis=0)
print(num_outliers)

tenure            0
MonthlyCharges    0
TotalCharges      0
dtype: int64


In [25]:
y_train.value_counts()

No     3870
Yes    1412
Name: Churn, dtype: int64

In [26]:
y_train.replace({'No':-1,'Yes':1}, inplace=True)
y_test.replace({'No':-1,'Yes':1}, inplace=True)

In [27]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score

kNN_model = KNN(n_neighbors=5).fit(x_train, y_train)
y_pred = kNN_model.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.750709823963657


In [28]:
from sklearn.linear_model import LinearRegression as LR

VIF_dict = dict()

for col in x_train.columns:
    model = LR().fit(x_train.drop([col],axis=1), x_train[col])
    r2 = model.score(x_train.drop([col],axis=1), x_train[col])
    VIF = 1/(1-r2)
    VIF_dict[col] = VIF
    
VIF_dict

{'SeniorCitizen': 1.164566118935981,
 'tenure': 7.5599350636664795,
 'MonthlyCharges': 880.311339051679,
 'TotalCharges': 10.692655159482971,
 'gender_Male': 1.002966246454167,
 'Partner_Yes': 1.4760590945986285,
 'Dependents_Yes': 1.404942289914632,
 'PhoneService_Yes': inf,
 'MultipleLines_No phone service': inf,
 'MultipleLines_Yes': 7.3673335853229185,
 'InternetService_Fiber optic': 150.66773780896852,
 'InternetService_No': inf,
 'OnlineSecurity_No internet service': inf,
 'OnlineSecurity_Yes': 6.460336851392473,
 'OnlineBackup_No internet service': inf,
 'OnlineBackup_Yes': 6.84876935883724,
 'DeviceProtection_No internet service': inf,
 'DeviceProtection_Yes': 7.031297884916123,
 'TechSupport_No internet service': inf,
 'TechSupport_Yes': 6.573717719718945,
 'StreamingTV_No internet service': inf,
 'StreamingTV_Yes': 24.569954930080414,
 'StreamingMovies_No internet service': inf,
 'StreamingMovies_Yes': 24.495991342926164,
 'Contract_One year': 1.6388192354367488,
 'Contract_T

In [29]:
remove_cols = [key for key in VIF_dict.keys() if VIF_dict[key] > 30]
remove_cols

['MonthlyCharges',
 'PhoneService_Yes',
 'MultipleLines_No phone service',
 'InternetService_Fiber optic',
 'InternetService_No',
 'OnlineSecurity_No internet service',
 'OnlineBackup_No internet service',
 'DeviceProtection_No internet service',
 'TechSupport_No internet service',
 'StreamingTV_No internet service',
 'StreamingMovies_No internet service']

In [30]:
x_train.drop(remove_cols, axis=1, inplace=True)
x_test.drop(remove_cols, axis=1, inplace=True)

cont_cols = list(set(cont_cols)-set(remove_cols))
cate_cols = list(set(cate_cols)-set(remove_cols))

In [32]:
skew_score = x_train[cont_cols].skew()
skew_score

TotalCharges    0.964029
tenure          0.238264
dtype: float64

In [35]:
biased_cols = x_train[cont_cols].columns[skew_score.abs() > 1.5]
x_train[biased_cols] = np.log10(x_train[biased_cols] - x_train[biased_cols].min()+1)

In [36]:
from sklearn.preprocessing import MinMaxScaler

MMS = MinMaxScaler().fit(x_train)
s_x_train = pd.DataFrame(MMS.transform(x_train), columns = x_train.columns)
s_x_test = pd.DataFrame(MMS.transform(x_test), columns = x_test.columns)

s_x_train.head()

,SeniorCitizen,tenure,TotalCharges,gender_Male,Partner_Yes,...,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1.0,0.583333,0.529297,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0
1,0.0,0.513889,0.085120,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
2,0.0,0.013889,0.000168,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0
3,0.0,0.319444,0.225258,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0
4,0.0,0.722222,0.123861,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0


In [37]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LGR

from sklearn.model_selection import ParameterGrid as PG
from sklearn.metrics import f1_score, classification_report
from sklearn.feature_selection import f_classif, chi2

MPG = dict() # model parameter grid

MPG[RFC] = PG({'max_depth':[2,3,4,5,6],
               'n_estimators':[5, 10, 25, 50, 75, 100]})
MPG[DTC] = PG({'max_depth':[2,3,4,5,6],
               'min_samples_leaf':[1,2,3,4,5,6]})
MPG[LGR] = PG({'C':[0.0001, 0.005, 0.001, 0.005, 0.01, 0.5, 0.1],
               'max_iter':[10,100,1000,10000,100000]})
MPG[KNN] = PG({'n_neighbors':[3,5,7,9,11]})

In [38]:
cont_cols_pval = f_classif(s_x_train[cont_cols],y_train)[1]
cate_cols_pval = chi2(s_x_train[cate_cols],y_train)[1]

cont_pvals = pd.Series(cont_cols_pval, index=cont_cols)
cate_pvals = pd.Series(cate_cols_pval, index=cate_cols)

pvals = pd.concat([cont_pvals, cate_pvals])
pvals.sort_values(ascending=True ,inplace=True)

pvals

tenure                                   2.818166e-161
Contract_Two year                         1.919828e-82
PaymentMethod_Electronic check            2.377284e-74
TotalCharges                              1.189847e-51
Contract_One year                         1.617775e-29
SeniorCitizen                             1.504183e-27
TechSupport_Yes                           7.678418e-26
OnlineSecurity_Yes                        7.232845e-25
Dependents_Yes                            5.591139e-24
PaymentMethod_Credit card (automatic)     2.291404e-19
PaperlessBilling_Yes                      5.361269e-18
Partner_Yes                               1.835174e-16
PaymentMethod_Mailed check                2.584568e-09
OnlineBackup_Yes                          5.550739e-08
DeviceProtection_Yes                      5.066801e-05
StreamingTV_Yes                           2.043694e-04
StreamingMovies_Yes                       1.127826e-03
MultipleLines_Yes                         7.550717e-02
gender_Mal

In [40]:
best_score = 0
for k in range(len(s_x_train.columns),5,-1):
    print(k)
    ss_x_train = s_x_train[pvals.iloc[:k].index]
    ss_x_test = s_x_test[pvals.iloc[:k].index]
    for M in MPG.keys():
        for P in MPG[M]:
            model = M(**P).fit(ss_x_train, y_train)
            y_pred = model.predict(ss_x_test)
            score = f1_score(y_test, y_pred, average='weighted')
            
            if score > best_score:
                best_score = score.mean()
                best_model = M
                best_params = P
                best_feature = ss_x_train.columns
                report = classification_report(y_test, y_pred)

19
18
17
16
15
14
13
12
11
10
9
8
7
6


In [41]:
print(f'best_score : {best_score}')
print(f'best_model : {best_model}')
print(f'best_params : {best_params}')
print(f'best_features : {best_feature}')
print('='*50)
print(report)

best_score : 0.7885295940218972
best_model : <class 'sklearn.linear_model._logistic.LogisticRegression'>
best_params : {'C': 0.5, 'max_iter': 100}
best_features : Index(['tenure', 'Contract_Two year', 'PaymentMethod_Electronic check',
       'TotalCharges', 'Contract_One year', 'SeniorCitizen', 'TechSupport_Yes',
       'OnlineSecurity_Yes', 'Dependents_Yes',
       'PaymentMethod_Credit card (automatic)', 'PaperlessBilling_Yes',
       'Partner_Yes', 'PaymentMethod_Mailed check', 'OnlineBackup_Yes',
       'DeviceProtection_Yes', 'StreamingTV_Yes', 'StreamingMovies_Yes',
       'MultipleLines_Yes', 'gender_Male'],
      dtype='object')
              precision    recall  f1-score   support

          -1       0.84      0.90      0.87      1304
           1       0.64      0.49      0.56       457

    accuracy                           0.80      1761
   macro avg       0.74      0.70      0.71      1761
weighted avg       0.79      0.80      0.79      1761

